# Millionaire Odds vs. Hit by a Bus: An ESQL Analysis

## Python modules

In [ ]:
!pip cache purge
!pip uninstall -y elasticsearch
!pip install elasticsearch==8.14 pandas matplotlib spicy

## Loading Synthetic data

In [ ]:
import pandas as pd
import numpy as np
from elasticsearch import Elasticsearch, helpers

# Initialize Elasticsearch client
client = Elasticsearch(
    hosts="YOUR_DEPLOYMENT_CLOUD_ID",
    api_key="YOUR_DEPLOYMENT_API_KEY",
)


# Generate synthetic data with a highly skewed distribution
num_records = 500000
np.random.seed(42)  # Ensure reproducibility

# Generate net worth using a highly skewed distribution
ages = np.random.randint(20, 80, num_records)  # Random ages between 20 and 80
incomes = np.random.exponential(
    scale=10000, size=num_records
)  # Exponential distribution for income
# Use a more skewed distribution for net worth with a much larger range
net_worths = np.random.exponential(
    scale=100000000, size=num_records
)  # Extremely skewed net worth

# Scale up the net worths to reach up to $100 billion
net_worths = np.clip(net_worths, 0, 100000000000)

# Create DataFrame
df = pd.DataFrame(
    {
        "id": range(1, num_records + 1),
        "age": ages,
        "income": incomes,
        "net_worth": net_worths,
        "counter": range(1, num_records + 1),  # Add a counter field for pagination
    }
)

# Index the data into Elasticsearch
index_name = "raw_wealth_data_large"
client.indices.delete(index=index_name, ignore=[400, 404])
client.indices.create(index=index_name, ignore=400)


def generator(df):
    for index, row in df.iterrows():
        yield {"_index": index_name, "_source": row.to_dict()}


helpers.bulk(client, generator(df))

print("Data indexed successfully.")

## Pulling data with ES|QL and creating a dataframe

In [ ]:
import pandas as pd
import numpy as np
from elasticsearch import Elasticsearch, helpers
from io import StringIO
import matplotlib.pyplot as plt
from scipy.stats import pareto


# Function to execute ESQL query and fetch data in chunks
def execute_esql_query(query):
    response = client.esql.query(query=query, format="csv")
    return pd.read_csv(StringIO(response.body))


# Function to fetch paginated data using the counter field
def fetch_paginated_data(index, num_records, size=10000):
    all_data = pd.DataFrame()
    for start in range(1, num_records + 1, size):
        end = start + size - 1
        query = f"""
        FROM {index}
        | WHERE counter >= {start} AND counter <= {end}
        | limit {size}
        """
        data_chunk = execute_esql_query(query)
        all_data = pd.concat([all_data, data_chunk], ignore_index=True)
    return all_data


# Fetch all data using pagination and ESQL
num_records = 500000
all_data_df = fetch_paginated_data(index_name, num_records)
print(f"Total Data Retrieved: {len(all_data_df)} records")

# Fit a Pareto distribution to the data
shape, loc, scale = pareto.fit(all_data_df["net_worth"], floc=0)

# Calculate the probability density for each net worth
all_data_df["net_worth_probability"] = pareto.pdf(
    all_data_df["net_worth"], shape, loc=loc, scale=scale
)

# Normalize the probabilities to sum to 1
all_data_df["net_worth_probability"] /= all_data_df["net_worth_probability"].sum()

print("Data with Net Worth Probability:")
print(all_data_df.head())

# Find the Net Worth Corresponding to the Bus Hit Probability
target_probability = 0.0000181
cumulative_probability = all_data_df["net_worth_probability"].cumsum()
target_net_worth_df = all_data_df[cumulative_probability >= target_probability].head(1)
target_net_worth = target_net_worth_df["net_worth"].iloc[0]
print(f"Net Worth with Probability >= {target_probability}: {target_net_worth}")

# Plot the Net Worth Probability Distribution
plt.figure(figsize=(10, 6))
plt.hist(
    all_data_df["net_worth"],
    bins=100,
    density=True,
    alpha=0.6,
    color="g",
    label="Empirical Data",
)
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = pareto.pdf(x, shape, loc=loc, scale=scale)
plt.plot(x, p, "k", linewidth=2, label="Fitted Pareto Distribution")
plt.axhline(
    y=target_probability, color="r", linestyle="--", label="Bus Hit Probability"
)
plt.axvline(
    x=target_net_worth,
    color="g",
    linestyle="--",
    label=f"Net Worth = {target_net_worth:.2f}",
)
plt.xlabel("Net Worth")
plt.ylabel("Probability")
plt.title("Net Worth Probability Distribution")
plt.legend()
plt.grid(True)
plt.show()

## Fit Pareto Distribution

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import pareto

# Assuming all_data_df contains the fetched net worth data from Elasticsearch
# Fit a Pareto distribution to the data
shape, loc, scale = pareto.fit(all_data_df["net_worth"], floc=0)

# Plot the Net Worth Probability Distribution
plt.figure(figsize=(10, 6))

# Plot histogram of empirical net worth data
plt.hist(
    all_data_df["net_worth"],
    bins=100,
    density=True,
    alpha=0.6,
    color="g",
    label="Empirical Data",
)

# Plot fitted Pareto distribution
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = pareto.pdf(x, shape, loc=loc, scale=scale)
plt.plot(x, p, "k", linewidth=2, label="Fitted Pareto Distribution")

# Show the plot
plt.xlabel("Net Worth")
plt.ylabel("Probability")
plt.title("Net Worth Probability Distribution")
plt.legend()
plt.grid(True)
plt.show()